In [4]:
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians

# Import and Merge Files

In [ ]:
# CSV file paths
df1 = pd.read_csv("Jun_11_2019.csv")
df2 = pd.read_csv("Jul_15_2019.csv")
df3 = pd.read_csv("Aug_5_2019.csv")
df4 = pd.read_csv("Jun_5_2020.csv")
df5 = pd.read_csv("Jul_5_2020.csv")
df6 = pd.read_csv("Aug_11_2020.csv")

In [ ]:
# concat csv
merge = [df1, df2, df3, df4, df5, df6]
merged_df = pd.concat(merge).drop_duplicates()

In [ ]:
merged_df.to_csv("2019_2020_citibike_comparison.csv", index=False)

# Clean Data

In [5]:
# read in merged csv
df = pd.read_csv("2019_2020_citibike_comparison.csv")
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,139,2019-05-01 00:00:01.9010,2019-05-01 00:02:21.5170,447.0,8 Ave & W 52 St,40.763707,-73.985162,423.0,W 54 St & 9 Ave,40.765849,-73.986905,31170,Subscriber,1983,1
1,754,2019-05-01 00:00:03.0210,2019-05-01 00:12:37.6920,3258.0,W 27 St & 10 Ave,40.750182,-74.002184,3255.0,8 Ave & W 31 St,40.750585,-73.994685,25560,Customer,1969,0
2,2308,2019-05-01 00:00:04.6270,2019-05-01 00:38:33.1710,3093.0,N 6 St & Bedford Ave,40.717452,-73.958509,3676.0,Van Brunt St & Van Dyke St,40.675833,-74.014726,33369,Subscriber,1978,1
3,143,2019-05-01 00:00:19.3340,2019-05-01 00:02:42.5200,3486.0,Schermerhorn St & Bond St,40.688417,-73.984517,3412.0,Pacific St & Nevins St,40.685376,-73.983021,32041,Subscriber,1997,1
4,138,2019-05-01 00:00:22.1840,2019-05-01 00:02:40.6480,388.0,W 26 St & 10 Ave,40.749718,-74.002950,494.0,W 26 St & 8 Ave,40.747348,-73.997236,35237,Subscriber,1967,1


In [8]:
# drop rows where ride is less than 5 minutes (300 seconds)
df = df[df['tripduration'] > 300]
df.shape

(10127033, 15)

In [9]:
# drop rows where no ride occurred (start and end stations are the same) (300 seconds)
df = df[df['start station id'] != df['end station id']]
df.shape

(9699210, 15)

In [10]:
# drop rows where trip is over 24 hours (bike may have been stolen)
df = df[df["tripduration"] < 86400]
df.shape

(9694092, 15)

In [11]:
# drop null values
df = df.dropna()
df.shape

(9694050, 15)

In [12]:
# change datatype to int
df['start station id'] = df['start station id'].astype('int')

In [13]:
# change datatype to int
df['end station id'] = df['end station id'].astype('int')

In [14]:
# Add ride id column for ride count
df["rideid"] = df["start station id"].map(str) + "_" + df["end station id"].map(str)
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,rideid
1,754,2019-05-01 00:00:03.0210,2019-05-01 00:12:37.6920,3258,W 27 St & 10 Ave,40.750182,-74.002184,3255,8 Ave & W 31 St,40.750585,-73.994685,25560,Customer,1969,0,3258_3255
2,2308,2019-05-01 00:00:04.6270,2019-05-01 00:38:33.1710,3093,N 6 St & Bedford Ave,40.717452,-73.958509,3676,Van Brunt St & Van Dyke St,40.675833,-74.014726,33369,Subscriber,1978,1,3093_3676
5,1598,2019-05-01 00:00:47.5650,2019-05-01 00:27:26.4150,310,State St & Smith St,40.689269,-73.989129,3322,12 St & 4 Ave,40.668603,-73.990439,16614,Customer,1973,1,310_3322
6,514,2019-05-01 00:01:06.0520,2019-05-01 00:09:40.1670,439,E 4 St & 2 Ave,40.726281,-73.989780,336,Sullivan St & Washington Sq,40.730477,-73.999061,15160,Subscriber,1988,1,439_336
7,524,2019-05-01 00:01:21.0070,2019-05-01 00:10:05.9440,380,W 4 St & 7 Ave S,40.734011,-74.002939,439,E 4 St & 2 Ave,40.726281,-73.989780,16742,Subscriber,1964,1,380_439


#### Calculate distance between latitude and longitude in dataframe using geopy & geodesic

In [28]:
from geopy.distance import geodesic

def distance_calc (row):
    start = (row['start station latitude'], row['start station longitude'])
    stop = (row['end station latitude'], row['end station longitude'])

    return geodesic(start, stop).miles

In [29]:
# very slow - uncomment and run with caution
#df['distance'] = df.apply (lambda row: distance_calc (row),axis=1)
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,gender,rideid,age,distance
1,12.566667,2019-05-01 00:00:03.0210,2019-05-01 00:12:37.6920,3258,W 27 St & 10 Ave,40.750182,-74.002184,3255,8 Ave & W 31 St,40.750585,-73.994685,25560,Customer,0,3258_3255,50-64,0.394526
2,38.466667,2019-05-01 00:00:04.6270,2019-05-01 00:38:33.1710,3093,N 6 St & Bedford Ave,40.717452,-73.958509,3676,Van Brunt St & Van Dyke St,40.675833,-74.014726,33369,Subscriber,1,3093_3676,40-49,4.118716
5,26.633333,2019-05-01 00:00:47.5650,2019-05-01 00:27:26.4150,310,State St & Smith St,40.689269,-73.989129,3322,12 St & 4 Ave,40.668603,-73.990439,16614,Customer,1,310_3322,40-49,1.427683
6,8.566667,2019-05-01 00:01:06.0520,2019-05-01 00:09:40.1670,439,E 4 St & 2 Ave,40.726281,-73.989780,336,Sullivan St & Washington Sq,40.730477,-73.999061,15160,Subscriber,1,439_336,30-39,0.566724
7,8.733333,2019-05-01 00:01:21.0070,2019-05-01 00:10:05.9440,380,W 4 St & 7 Ave S,40.734011,-74.002939,439,E 4 St & 2 Ave,40.726281,-73.989780,16742,Subscriber,1,380_439,50-64,0.872716


#### Bin Birth Year

In [15]:
# Faulty birth dates included, will need to drop these
df['birth year'].min()

1873

In [16]:
df['birth year'].max()

2004

In [17]:
# drop rows with birth year less than 1930
df= df[df['birth year'] > 1930]
df.shape

(9688471, 16)

In [22]:
# Divide users into bins by birth year
bins = [1930, 1955, 1970, 1980, 1990, 2001, 2020] 
labels = ["65-90", "50-64", "40-49", "30-39", "19-29", "0-18"]

df["age"] = pd.cut(df["birth year"], bins, labels=labels)
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,rideid,Age,age
1,754,2019-05-01 00:00:03.0210,2019-05-01 00:12:37.6920,3258,W 27 St & 10 Ave,40.750182,-74.002184,3255,8 Ave & W 31 St,40.750585,-73.994685,25560,Customer,1969,0,3258_3255,50-64,50-64
2,2308,2019-05-01 00:00:04.6270,2019-05-01 00:38:33.1710,3093,N 6 St & Bedford Ave,40.717452,-73.958509,3676,Van Brunt St & Van Dyke St,40.675833,-74.014726,33369,Subscriber,1978,1,3093_3676,40-49,40-49
5,1598,2019-05-01 00:00:47.5650,2019-05-01 00:27:26.4150,310,State St & Smith St,40.689269,-73.989129,3322,12 St & 4 Ave,40.668603,-73.990439,16614,Customer,1973,1,310_3322,40-49,40-49
6,514,2019-05-01 00:01:06.0520,2019-05-01 00:09:40.1670,439,E 4 St & 2 Ave,40.726281,-73.989780,336,Sullivan St & Washington Sq,40.730477,-73.999061,15160,Subscriber,1988,1,439_336,30-39,30-39
7,524,2019-05-01 00:01:21.0070,2019-05-01 00:10:05.9440,380,W 4 St & 7 Ave S,40.734011,-74.002939,439,E 4 St & 2 Ave,40.726281,-73.989780,16742,Subscriber,1964,1,380_439,50-64,50-64


#### clean data con't: 

In [23]:
# Delete columns
del df["birth year"]
del df["Age"]

In [25]:
# change trip duration column to minutes
df["tripduration"] = [i/60 for i in df["tripduration"]]

### Export clean data

In [31]:
df.to_csv("2019_2020_citibike_comparison.csv", index=False)